In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import re
import pathlib
import pprint
from io import StringIO
pp = pprint.PrettyPrinter(indent=2, compact=False, width=80)

from typing import Union, Tuple, IO

# Motivation

How many times have you seen an API like this?

In [3]:
class DataFrame:
    pass

In [4]:
def read_csv(filepath_or_buffer: Union[str, IO[str]], *args, **kwargs) -> DataFrame:
    """
    If filepath_or_buffer is a str it is interpreted as a file path,
    if it is a stream it is read directly as text
    """
    pass

Or like this?

In [5]:
def somefunc(return_extra: bool=False) -> Union[str, Tuple[str]]:
    rv = 'Foo'
    if return_extra:
        return rv, 'extra'
    
    return rv

print(somefunc())
print(somefunc(return_extra=True))

Foo
('Foo', 'extra')


# Alternate Constructors


In [6]:
from itertools import chain

to_chain = [(1, 2), (3, 4), (5, 6, 7)]

In [7]:
c = chain(*to_chain)
print(list(c))

[1, 2, 3, 4, 5, 6, 7]


In [8]:
list(chain.from_iterable(to_chain))

[1, 2, 3, 4, 5, 6, 7]

## API

- `chain(*args: Iterable) -> chain` - Create a chain from many iterables.
- `chain.from_iterable(iterable: Iterable[Iterable]) -> chain` - Create a chain from an iterable of iterables.

# `variants`

## Desired API
- `print_text(txt: str)` - Print text passed to this function

- `print_text.from_stream(sobj: IO[str])` - Print text from a stream

- `print_text.from_filepath(*path_components: str)` - Open a file on disk and print its contents

In [9]:
import variants

@variants.primary
def print_text(txt):
    """Prints any text passed to this function"""
    print(txt)

In [10]:
@print_text.variant('from_stream')
def print_text(sobj):
    """Read text from a stream and print it"""
    print_text(sobj.read())

In [11]:
@print_text.variant('from_filepath')
def print_text(*path_components):
    """Open the file specified by `path_components` and print the contents"""
    fpath = pathlib.Path(*path_components)
    with open(fpath, 'r') as f:
        print_text.from_stream(f)

## Example use

In [12]:
print_text('Hello, world')

Hello, world


In [13]:
print_text.from_stream(StringIO('Hello, world! This is from a stream!'))

Hello, world! This is from a stream!


In [14]:
print_text.from_filepath('extras/hello_world.txt')

Hello, world! This is from a file!



In [15]:
print_text.from_filepath('extras', 'hello_world.txt')

Hello, world! This is from a file!



# Why use variants?

- Syntactically marks relatedness, as compared to "pseudo-namespacing" using `_`:

In [16]:
def print_text_from_filepath(fpath):
    print_text.from_filepath(fpath)
    
print_text_from_filepath('extras/hello_world.txt')

Hello, world! This is from a file!



- Variants are namespaced *under* the primary variant:
    - Tab completion helps discovery of variant functions
    - "Top level" API is kept as clean as possible

## Autodocument with sphinx

Using the `sphinx_autodoc_variants` sphinx extension:

```rst
.. automodule:: text_print
    :members:
```

![text_print module documentation](images/sphinx_docs_image.png)

## Reasons to use `variants`

### Explicit dispatch

In [17]:
import requests

In [18]:
@print_text.variant('from_url')
def print_text(url):
    r = requests.get(url)
    print_text(r.text)

In [19]:
@print_text.variant('from_url')
def print_text(url):
    try:
        r = requests.get(url)
    except Exception:
        print_text("Hello, world! (from url)")
    print(r.text)

In [20]:
print_text('Hello, world!')
print_text.from_filepath('extras/hello_world.txt')
print_text.from_url('https://ganssle.io/files/hello_world.txt')

Hello, world!
Hello, world! This is from a file!

Hello, world! (from url)


## Why not use singledispatch?

In [21]:
from functools import singledispatch

from pathlib import Path

class Url:
    def __init__(self, url):
        self.url = url
    
    def __repr__(self):
        return f"{self.__class__.__name__}('{self.url}')"
    
    def __str__(self):
        return self.url

In [22]:
@singledispatch
def print_text_sd(txt):
    print(txt)

@print_text_sd.register(Url)
def _(url):
    print_text.from_url(url.url)

@print_text_sd.register(Path)
def _(path):
    print_text.from_filepath(path)

In [23]:
print_text_sd('Hello, world!')
print_text_sd(Path('extras/hello_world.txt'))
print_text_sd(Url('https://ganssle.io/files/hello_world.txt'))

Hello, world!
Hello, world! This is from a file!

Hello, world! (from url)


### Can combine with singledispatch

In [24]:
from functools import singledispatch

@variants.primary
@singledispatch
def add(x, y):
    return x + y

@add.variant('from_list')
@add.register(list)
def add(x, y):
    return x + [y]

In [25]:
print('1 + 2 = %d' % add(1, 2))
print('[1] + 2 = %s' % add([1], 2))

1 + 2 = 3
[1] + 2 = [1, 2]


In [26]:
print('add.from_list([1], 2) = %s' % add.from_list([1], 2))

add.from_list([1], 2) = [1, 2]


In [27]:
try:
    add.from_list(1, 3)
except Exception as e:
    print('add.from_list(1, 2): %s' % repr(e))

add.from_list(1, 2): TypeError("unsupported operand type(s) for +: 'int' and 'list'",)


## Reasons to use `variants`

### Variation in return type

In [28]:
from typing import Iterator, List

In [29]:
from datetime import date, timedelta

@variants.primary
def date_range(dt_start: date, dt_end: date) -> Iterator[date]:
    dt = dt_start
    step = timedelta(days=1)
    while dt < dt_end:
        yield dt
        dt += step
    

In [30]:
date_range(date(2018, 1, 14), date(2018, 1, 18))


<generator object date_range at 0x7f1b8be1bba0>

In [31]:
@date_range.variant('eager')
def date_range(dt_start: date, dt_end: date) -> List[date]:
    return list(date_range(dt_start, dt_end))

In [32]:
date_range.eager(date(2018, 1, 14), date(2018, 1, 18))

[datetime.date(2018, 1, 14),
 datetime.date(2018, 1, 15),
 datetime.date(2018, 1, 16),
 datetime.date(2018, 1, 17)]

## Reasons to use `variants`

## Variation in async behavior

In [33]:
import asyncio
import time

@variants.primary
async def myfunc(n):
    for i in range(n):
        yield i
        await asyncio.sleep(0.5)
        
@myfunc.variant('sync')
def myfunc(n):
    for i in range(n):
        yield i
        time.sleep(0.5)

In [34]:
myfunc(4)

<async_generator object myfunc at 0x7f1b8be25868>

In [35]:
myfunc.sync(4)

<generator object myfunc at 0x7f1b8be5a468>

## Variation in caching behavior

In [36]:
from functools import lru_cache

@variants.primary
@lru_cache()
def get_config():
    return get_config.nocache()

@get_config.variant('nocache')
def get_config():
    print("Retrieving configuration!")
    return {
        'a1': 'value',
        'b': 12348
    }


In [37]:
a = get_config()

Retrieving configuration!


In [38]:
b = get_config()

In [39]:
c = get_config.nocache()

Retrieving configuration!


# Project Links

- Find the project:
  - PyPI: `variants`: https://pypi.org/project/variants/
  - Github: https://github.com/python-variants/variants

## To Do:

- [Preserve introspection](https://github.com/python-variants/variants/issues/7)
- [Add compiled backend](https://github.com/python-variants/variants/issues/26)